In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-jax/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [134]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-jax',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 600 commands: 
python train_plife_prompt_sa.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/plife_var/seed=0_n_particles=1000_n_colors=2_render_radius=0.005_mutate_params='mass+c_dist+x_dist'"                              --n_particles=1000 --n_colors=2 --x_dist_bins=9 --prompt="" --render_radius=0.005 --render_sharpness=20.0 --rollout_steps=4000 --bs=4 --n_iters=256 --mr=1.0 --anneal_prob_end=1.0 --mutate_params="mass+c_dist+x_dist"                             
python train_plife_prompt_sa.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/plife_var/seed=0_n_particles=1000_n_colors=2_render_radius=0.005_mutate_params=''"                                                --n_particles=1000 --n_colors=2 --x_dist_bins=9 --prompt="" --render_radius=0.005 --render_sharpness=20.0 --rollout_steps=4000 --bs=4 --n_iters=256 --mr=1.0 --anneal_prob_end=1.0 --mutate_params=""                                               
python train_plife_prompt

In [135]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26


In [136]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [137]:
print('\n'.join(txt.split('\n')[:24]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-jax
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/job_00003.status
python train_plife_prompt_sa.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/plife_var/seed=0_n_particles=1000_n_colors=2_render_radius=0.005_mutate_params='alpha+c_dist+x_dist'"                             --n_particles=1000 --n_colors=2 --x_dist_bins=9 --prompt="" --render_radius=0.005 --render_sharpness=20.0 --rollout_steps=4000 --bs=4 --n_iters=256 --mr=1.0 --anneal_prob_end=1.0 --mutate_params="alpha+c_dist+x_dist"                             &> /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/job_00003.status

touch /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/job_00007.status
python train_plife_prompt_sa.py --seed=0 --save_dir="/home/aka

In [138]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_21_08_01_26/gpu_7.sh


# Track Experiment

In [253]:
from collections import defaultdict

statuses = []
for job_id in range(len(commands)):
    try:
        with open(f"{log_dir}/job_{job_id:05d}.status", "r") as f:
            status = int(f.read().strip())
            statuses.append(status)
    except Exception as e:
        pass
        # print(e)
        # break
statuses = np.array(statuses)

In [254]:
(statuses==0).sum(), len(commands)

(600, 600)

In [255]:
for job_id in list(range(len(commands)))[-40:]:
    print()
    print(f'------------------------------------------------------ {job_id} ------------------------------------------------------')
    print()
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        # pass
        # break
        print(e)


------------------------------------------------------ 560 ------------------------------------------------------

self.pid = _fork_exec(

------------------------------------------------------ 561 ------------------------------------------------------

self.pid = _fork_exec(

------------------------------------------------------ 562 ------------------------------------------------------

self.pid = _fork_exec(

------------------------------------------------------ 563 ------------------------------------------------------

self.pid = _fork_exec(

------------------------------------------------------ 564 ------------------------------------------------------

self.pid = _fork_exec(

------------------------------------------------------ 565 ------------------------------------------------------

self.pid = _fork_exec(

------------------------------------------------------ 566 ------------------------------------------------------

self.pid = _fork_exec(

--------------------------